In [1]:
import sys
import os
# add the folder where util.py lives
sys.path.append(os.path.join(os.getcwd(), 'fit3182-a2'))
sys.path.insert(0, os.path.abspath(os.getcwd()))

from pyspark.sql.functions import udf, col, window
from pyspark.sql.types import StringType
from util import kafkaConsumer
import pandas as pd
from Operations import SparkInst, DbWriter, ConsoleWriter
from pyspark.sql.functions import lit
from pyspark.sql.streaming import StreamingQueryException

In [2]:
spark_job=SparkInst("AWAS SYSTEM", 5, "violations")
hostip = "172.22.32.1"
port_spark = 9092
port_mongo = 27017

In [3]:
import pandas as pd

df_pd = pd.read_csv("data/camera.csv")
if '_id' in df_pd.columns:
    df_pd.drop(columns=['_id'], inplace=True)
spark_df = spark_job.get_session().createDataFrame(df_pd)

# Step 3: Broadcast your speed limit map
speed_limit_map = {row['camera_id']: row['speed_limit'] for row in spark_df.select("camera_id", "speed_limit").collect()}
broadcast_map = spark_job.essentialData_broadcast(spark_df)
# Step 4: Define your UDF using broadcast variable
def mark_speeding(camera_id:str, speed:float, ops:str):
    limit = broadcast_map.value.get(camera_id)
    if limit is not None and ops == "instant":
        return "INSTANT_VIOLATION" if speed > limit else None
    elif limit is not None and ops == "average":
        return "AVERAGE_VIOLATION" if speed > limit else None
    return "NONE"

speeding_udf = udf(mark_speeding, StringType())

# Step 5: Apply UDF to each streaming dataframe
def add_speed_flag(df, ops: str):
    return df.withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed_reading"), lit(ops)))


/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
from pyspark.sql.functions import expr, col

# Attach Kafka streams
stream_a = spark_job.attach_kafka_stream("camera_event_a", hostip, "10 hours")
stream_b = spark_job.attach_kafka_stream("camera_event_b", hostip, "10 hours")
stream_c = spark_job.attach_kafka_stream("camera_event_c", hostip, "10 hours")

# Drop unnecessary columns and apply speed flag
stream_a_flagged = add_speed_flag(stream_a.drop("event_id"), "instant")
stream_b_flagged = add_speed_flag(stream_b.drop("event_id"), "instant")
stream_c_flagged = add_speed_flag(stream_c.drop("event_id"), "instant")

# Duplicate stream_b for two separate joins
dup1_b = stream_b_flagged
dup2_b = stream_b_flagged

# Rename timestamp and camera_id for join logic
stream_a_renamed = stream_a_flagged \
    .withColumnRenamed("timestamp", "timestamp_start") \
    .withColumnRenamed("camera_id", "camera_id_start")

stream_b1_renamed = dup1_b \
    .withColumnRenamed("timestamp", "timestamp_end") \
    .withColumnRenamed("camera_id", "camera_id_end")

stream_c_renamed = stream_c_flagged \
    .withColumnRenamed("timestamp", "timestamp_end") \
    .withColumnRenamed("camera_id", "camera_id_end")

stream_b2_renamed = dup2_b \
    .withColumnRenamed("timestamp", "timestamp_start") \
    .withColumnRenamed("camera_id", "camera_id_start")

# Apply watermarks
# stream_a_watermarked = stream_a_renamed.withWatermark("sent_at", "10 hours")
# stream_b1_watermarked = stream_b1_renamed.withWatermark("sent_at", "10 hours")
# stream_c_watermarked = stream_c_renamed.withWatermark("sent_at", "10 hours")
# stream_b2_watermarked = stream_b2_renamed.withWatermark("sent_at", "10 hours")

# Join A and B
ab_join = stream_b1_renamed.alias("b").join(
    stream_a_renamed.alias("a"),
    expr("""
      a.car_plate = b.car_plate AND
      b.timestamp_end BETWEEN a.timestamp_start AND a.timestamp_start + INTERVAL 10 MINUTES
    """),
    "inner"
)

# ab_join = stream_b1_renamed.alias("b").join(
#     stream_a_renamed.alias("a"),
#     (
#         (col("a.car_plate") == col("b.car_plate")) &
#         (col("a.timestamp_start") > col("b.timestamp_end")) &
#         (col("a.timestamp_start") <= col("b.timestamp_end") + expr("interval 10 minutes"))
#     ),
#     "inner"
# )

ab_join = ab_join.withColumn(
    "avg_speed_reading",
    (col("a.speed_reading") + col("b.speed_reading")) / 2
).withColumn(
    "speed_flag",
    speeding_udf(col("a.camera_id_start"), col("a.speed_reading"), lit("average"))
)

# Join B and C
bc_join = stream_b2_renamed.alias("b").join(
    stream_c_renamed.alias("c"),
    expr("""
      b.car_plate = c.car_plate AND
      c.timestamp_end BETWEEN b.timestamp_start AND b.timestamp_start + INTERVAL 10 MINUTES
    """),
    "inner"
)

# Join B and C
# bc_join = stream_b2_renamed.alias("b").join(
#     stream_c_renamed.alias("c"),
#     (
#         (col("b.car_plate") == col("c.car_plate")) &
#         (col("c.timestamp_end") > col("b.timestamp_start")) &
#         (col("c.timestamp_end") <= col("b.timestamp_start") + expr("interval 10 minutes"))
#     ),
#     "inner"
# )

bc_join = bc_join.withColumn(
    "avg_speed_reading",
    (col("b.speed_reading") + col("c.speed_reading")) / 2
).withColumn(
    "speed_flag",
    speeding_udf(col("c.camera_id_end"), col("c.speed_reading"), lit("average"))
)

res = ab_join.union(bc_join)
res = res.dropDuplicates()
# ab_join = (
#     stream_a_watermarked.alias("A")
#     .join(
#         stream_b1_watermarked.alias("B"),
#         (col("A.car_plate") == col("B.car_plate")),
#         how="inner"
#     )
#     .select(
#         col("A.*"),
#         col("B.batch_id").alias("batch_id_B"),
#         col("B.car_plate").alias("car_plate_B"),
#         col("B.camera_id").alias("camera_id_B"),
#         col("B.timestamp").alias("timestamp_B"),
#         col("B.speed_reading").alias("speed_reading_B"),
#         col("B.sent_at").alias("timestamp_B"),
#     )
# )

# ab_join = (
#     stream_a_watermarked.alias("A")
#       .join(
#           stream_b1_watermarked.alias("B"),
#           (col("A.car_plate") == col("B.car_plate"))
#           & (col("A.timestamp") < col("B.timestamp"))
#           & (col("A.timestamp") + expr("INTERVAL 5 MINUTES") >= col("B.timestamp")),
#           how="inner"
#       )
#       .withColumn(
#           "avg_measure_speed",
#           (col("A.speed_reading") + col("B.speed_reading")) / 2
#       )
#       .withColumn(
#           "speed_flag",
#           speeding_udf(col("B.camera_id"), col("avg_measure_speed"), lit("average"))
#       )
# )

# # Join B and C on car_plate and timestamp condition
# bc_join = stream_b2_watermarked.join(
#     stream_c_watermarked,
#     expr("""
#         car_plate = car_plate AND
#         timestamp_c > timestamp_b2 AND
#         timestamp_c <= timestamp_b2 + interval 10 minutes
#     """),
#     "inner"
# ).withColumn(
#     "avg_measure_speed",
#     (col("measure_speed_a") + col("measure_speed_b")) / 2
# ).withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed_reading"), "average"))


In [11]:
spark = ab_join
mongo_uri = f"mongodb://{hostip}:{port_mongo}"
db_name   = "fit3182_db"
coll_name = "Violation"

# writer = DbWriter(
#     spark,
#     mongo_uri,
#     mongo_db=   db_name,
#     mongo_collection= coll_name
# )

# writer_stream = (
#     ab_join
#       .writeStream.format("Console")
#       .option("checkpointLocation", "./stream_checkpoints")
#       .outputMode("append")
#       .foreach(ConsoleWriter())     
# )

writer_stream = (
    res
      .select("*")
      .writeStream
      .format("console")
      .outputMode("append")
      .option("checkpointLocation", "./stream_checkpoints") 
)
try:
    query = writer_stream.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopped query')
except StreamingQueryException as exc:
    print(exc)
finally:
    query.stop()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Interrupted by CTRL-C. Stopped query


In [6]:
# ab_join.union(bc_join, window(col("timestamp"), "24 hours")).dropna().select()
